In [1]:
# Import Necessary Modules (some may not be immediately necessary )
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import itertools
import re
from pubchemprops.pubchemprops import get_second_layer_props, get_cid_by_name
import pubchempy as pcp
import time

# Suppress dumb warnings
pd.options.mode.chained_assignment = None  # default='warn'

# Data Query from OpenFDA

In [ ]:
#get Data from NDC DB


#define function to obtain chemical name, dosage form and melting point
def getAPI(API_name, n_lim = 1000, skip = 0):
    #Enpoint to search for API from label
    df_list = []
    url_root = "https://api.fda.gov/drug/ndc.json?"
    APIkey = "api_key=ieSIcZGsM6K0XVG6kL7UiQb4Yw3cDN1z2bwO4XPl"
    while True:
        query = "{}{}&search=active_ingredients.name:{}&limit={}&skip={}".format(url_root, APIkey, API_name, n_lim, skip)
        try:
            json_result = requests.get(query).json()['results']
            json_dict = {'product_ndc':[i['product_ndc'] for i in json_result],
                        'API':[i['active_ingredients'][0]['name'] for i in json_result],
                        'searched_API':[API_name for i in json_result],
                        'Dosage_strength':[i['active_ingredients'][0]['strength'] for i in json_result],
                        'Dosage_form':[i['dosage_form'] for i in json_result]}
            
            df_list.append(pd.DataFrame(json_dict))
            skip += n_lim
            
        except:
            break
    if len(df_list)>0:
        return pd.concat(df_list, axis=0, ignore_index=True)
    else:
        return pd.DataFrame.from_dict({'product_ndc': ['Not_found'],
                        'API':['Not_found'],
                        'searched_API':[API_name],
                        'Dosage_strength':['Not_found'],
                        'Dosage_form':['Not_found']},orient='columns')

In [ ]:
ess_drugs = pd.read_csv(r"C:\Users\nvu\Desktop\Data_Projects\Drug_Prices\API_list_600.csv", header=None)
api_list = ",".join(list(ess_drugs.iloc[:,0].values)).lower().strip().split(',')

fda_queried = []

for api_name in api_list:
    x_api = getAPI(api_name)
    time.sleep(0.35) #wait for API to send back request
    fda_queried.append(x_api)
    

In [ ]:
len(fda_queried)

In [ ]:
fda_queried_df = pd.concat(fda_queried, axis=0, ignore_index=True)

In [ ]:
fda_queried_df.head(2)

In [ ]:
notfound_df = fda_queried_df[fda_queried_df["API"]=="Not_found"]

notfound_df["searched_API"].value_counts()

In [ ]:
fda_queried_df.to_csv("NDC_Drug_FDA_raw.csv")

# Load saved queried Data and perform Data cleaning

In [ ]:
fda_queried_df = pd.read_csv("NDC_Drug_FDA_raw.csv")

In [ ]:
dosage_forms_type = json.loads(fda_queried_df.Dosage_form.value_counts().to_json())

In [ ]:
def get_concentration(string_info_in, para_out = "All"):
    conc_pattern = r'(\S+)\s(\S+)[/](\S+)'
    
    reg_result = re.findall(conc_pattern, string_info_in, re.IGNORECASE)
    
    if len(reg_result)==0:
        return (None, None, None)
    else:
        if para_out=="All":
            return reg_result[0]
        elif para_out=="Strength_number":
            return reg_result[0][0]
        elif para_out=="Strength_unit":
            return reg_result[0][1]
        elif para_out=="per_unit":
            return reg_result[0][-1]


In [ ]:
fda_queried_df_clean = fda_queried_df.loc[fda_queried_df["API"]!="Not_found", :]

fda_queried_df_clean["strength_num"] = fda_queried_df_clean.apply(lambda x: get_concentration(x["Dosage_strength"], para_out="Strength_number"), axis=1)

In [ ]:
fda_queried_df_clean["Strength_unit"] = fda_queried_df_clean.apply(lambda x: get_concentration(x["Dosage_strength"], para_out="Strength_unit"), axis=1)

fda_queried_df_clean["per_unit"] = fda_queried_df_clean.apply(lambda x: get_concentration(x["Dosage_strength"], para_out="per_unit"), axis=1)

In [ ]:
def check_unit(unit_in):
    if unit_in in {'mg', 'kg', 'g','ug'}:
        return True
    else: 
        return False
    
def conversion_to_mg(number, unit, func=check_unit):
    conversion_table = {
        'mg': 1,
        'kg': 1000*1000,
        'g': 1000,
        'ug': 1/1000,
    }
    if func(unit) is True:
        return float(number)*conversion_table[unit]
    else:
        try:
            return float(unit)
        except:
            return 0

In [ ]:
def parse_unit(unit_in, para = "num"):
    def isfloat(value):
        try:
            float(value)
            return True
        except ValueError:
            return False
    
    patrn = r'([0-9.]+)?([A-Za-z%]+)$'
    
    if isfloat(unit_in):
        return float(unit_in)
    else:
        reggex_result = re.findall(patrn, unit_in, re.IGNORECASE)
    
    if para=="num":
        try:
            if reggex_result[0][0] == '':
                return 1.0
            else:
                return float(reggex_result[0][0])
        except:
            return 0.0
    elif para=="unit":
        try:
            return reggex_result[0][1]
        except:
            return ''
    

In [ ]:
fda_queried_df_clean["unit_num"] = fda_queried_df_clean.apply(lambda x: parse_unit(x["per_unit"]), axis=1)

fda_queried_df_clean["unit_str"] = fda_queried_df_clean.apply(lambda x: parse_unit(x["per_unit"], para='unit'), axis=1)

In [ ]:
fda_queried_df_clean.head()

In [ ]:
fda_queried_df_clean["valid_row"] = fda_queried_df_clean.apply(lambda x: check_unit(x['Strength_unit']), axis=1)

fda_queried_df_clean = fda_queried_df_clean.loc[fda_queried_df_clean["valid_row"]==True, :]

In [ ]:
fda_queried_df_clean.unit_str.value_counts()

In [ ]:
fda_queried_df_clean.head(3)

In [ ]:
fda_queried_df_clean["strength_in_mg"] = fda_queried_df_clean.apply(lambda x: conversion_to_mg(number=x['strength_num'], unit=x['Strength_unit']), axis=1)

fda_queried_df_clean["unit_in_mg"] = fda_queried_df_clean.apply(lambda x: conversion_to_mg(number=x['unit_num'], unit=x['unit_str']), axis=1)

fda_queried_df_clean_unit = fda_queried_df_clean.loc[fda_queried_df_clean['unit_in_mg']>0, :]

fda_queried_df_clean_unit['strength_in_mg_per_1unit'] = fda_queried_df_clean_unit['strength_in_mg']/fda_queried_df_clean_unit['unit_in_mg']

In [ ]:
fda_queried_df_clean_unit[fda_queried_df_clean_unit.unit_str == 1].head()

In [ ]:
fda_queried_df_clean_unit.unit_str.value_counts()

In [ ]:
fda_queried_df_clean_unit = fda_queried_df_clean_unit.drop(axis=1, columns=['Unnamed: 0', 'unit_num', 'unit_str', 'valid_row'])

### Save clean-up table of all APIs


In [ ]:
fda_queried_df_clean_unit.to_csv("fda_queried_df_cleaned.csv")

In [ ]:
fda_queried_df_clean_unit.head(2)

# Read in cleaned Data Set to get PubChem Data

In [2]:
fda_queried_df_clean_unit = pd.read_csv('fda_queried_df_cleaned.csv')

In [3]:
fda_queried_df_clean_unit.head(3)

def is_tablet(dosage_form_str):
    
    if 'TABLET' in dosage_form_str.upper():
        return True
    else:
        return False

fda_queried_df_clean_unit['is_tablet'] = fda_queried_df_clean_unit['Dosage_form'].apply(lambda x: is_tablet(x))

fda_queried_df_clean_unit = fda_queried_df_clean_unit.loc[fda_queried_df_clean_unit['is_tablet'] == True, :]

In [41]:
fda_queried_df_clean_unit.to_csv("fda_tablet_Only_data.csv")

## Look for chemical info from Pubchem

In [24]:
def get_pubChem_info(name_in, para_str = 'MP, LogP' ):
    """
    para_str: "MP, LogP"
    """
    
    params_dict = dict(zip(['MP', 'LOGP'], ['Melting Point','LogP']))
    
    
    params_list = [params_dict[i.strip().upper()] for i in para_str.split(',')]
    
    def get_attributes(name, params):
        second_layer = get_second_layer_props(name, params)
        dict_out = dict(zip([i for i in params], [second_layer[i][0]['Value'] for i in params]))
        # dict_out["Mol_name"] = name
        return list(dict_out.values())

    try:
        return(get_attributes(name_in, params_list))
            
    except:
        try:
            synonyms_ls = pcp.get_synonyms(name_in, 'name')[0]["Synonym"]
            for s in synonyms_ls[:5]:
                try:
                    return(get_attributes(s, params_list))
                except:
                    if s == synonyms_ls[4]:
                        break
                    else:
                        print(s)
                        continue
            return [np.nan, np.nan]
        except:
            return [np.nan, np.nan]

            

In [33]:
mol_names = np.array(list(set(fda_queried_df_clean_unit['searched_API'].values)))

finalData_df = pd.DataFrame({'Molecule':mol_names})


In [35]:
finalData_df.shape

(468, 1)

In [ ]:
finalData_df["pubChem"] = finalData_df.apply(lambda x: get_pubChem_info(x['Molecule']), axis=1)

# x_mols_test = mol_names[:5]
# x_test = pd.DataFrame({'Molecule':x_mols_test})
# x_test["pubchem"] = x_test.apply(lambda x: get_pubChem_info(x['Molecule']), axis=1)

In [37]:
finalData_df.head(4)

,Molecule,pubChem
0,acetaminophen,[{'StringWithMarkup': [{'String': '336 to 342 ...
1,methamphetamine,"[nan, nan]"
2,acyclovir,"[{'Number': [255], 'Unit': '°C'}, {'Number': [..."
3,vilazodone,[{'StringWithMarkup': [{'String': 'MP: 227-279...


In [38]:
finalData_df['MP'] = finalData_df['pubChem'].apply(lambda x: x[0])
finalData_df['LogP'] = finalData_df['pubChem'].apply(lambda x: x[1])

In [39]:
finalData_df.head(3)

,Molecule,pubChem,MP,LogP
0,acetaminophen,[{'StringWithMarkup': [{'String': '336 to 342 ...,{'StringWithMarkup': [{'String': '336 to 342 °...,{'Number': [0.91]}
1,methamphetamine,"[nan, nan]",NaN,NaN
2,acyclovir,"[{'Number': [255], 'Unit': '°C'}, {'Number': [...","{'Number': [255], 'Unit': '°C'}",{'Number': [-1.76]}


In [40]:
finalData_df.to_csv("pubchem_MP_logP.csv")

# Load in cleaned molecules data and Pubchem Data

In [42]:
master_dt = pd.read_csv('fda_tablet_Only_data.csv')
pubchem_dt = pd.read_csv('pubchem_MP_logP.csv')

In [45]:
master_dt = master_dt.drop(axis=1, columns=master_dt.columns[:2])

In [46]:
pubchem_dt = pubchem_dt.drop(axis=1, columns=pubchem_dt.columns[:1])

In [50]:
master_dt.columns

Index(['product_ndc', 'API', 'searched_API', 'Dosage_strength', 'Dosage_form',
       'strength_num', 'Strength_unit', 'per_unit', 'strength_in_mg',
       'unit_in_mg', 'strength_in_mg_per_1unit', 'is_tablet'],
      dtype='object')

In [95]:

result = master_dt.set_index('searched_API').join(pubchem_dt.set_index('Molecule'), how='outer',)

In [117]:
result_found = result.dropna(axis=0)

In [119]:
result_found.columns

Index(['product_ndc', 'API', 'Dosage_strength', 'Dosage_form', 'strength_num',
       'Strength_unit', 'per_unit', 'strength_in_mg', 'unit_in_mg',
       'strength_in_mg_per_1unit', 'is_tablet', 'pubChem', 'MP', 'LogP'],
      dtype='object')

In [145]:
def parse_number(string_in):
    
    num_pattern = r"(-?\d+(\.\d+)?)"
    deg_pattern = r"([°|\s|'][F|C|K])"
    
    num_regex = re.search(num_pattern, string_in, re.IGNORECASE)
    unit_regex = re.search(deg_pattern, string_in)
    
    if num_regex != None:
        num = float(num_regex.group(0))
    else:
        num = np.nan
    if unit_regex != None:
        uni = unit_regex.group(0)
    else:
        uni = "_"

    if 'F' in uni:
        cor_num = (num-32)*(5/9)
    elif 'K' in uni:
        cor_num = num-273.15
    else:
        cor_num = num
    
    return (cor_num, uni)
        
    

In [149]:
parse_number(r"{'StringWithMarkup': [{'String': '342 °K (NTP, 1992)'}]}")[0]

68.85000000000002

In [150]:
result_found['MP_num_C'] = result_found['MP'].apply(lambda x: parse_number(x)[0])
result_found['logP_num'] = result_found['LogP'].apply(lambda x: parse_number(x)[0])

In [151]:
result_found.head(3)

,product_ndc,API,Dosage_strength,Dosage_form,strength_num,Strength_unit,per_unit,strength_in_mg,unit_in_mg,strength_in_mg_per_1unit,is_tablet,pubChem,MP,LogP,MP_num_C,logP_num
abacavir,65862-073,ABACAVIR SULFATE,300 mg/1,"TABLET, FILM COATED",300.0,mg,1,300.0,1.0,300.0,True,"[{'Number': [165], 'Unit': '°C'}, {'Number': [...","{'Number': [165], 'Unit': '°C'}",{'Number': [1.2]},165.0,1.2
abacavir,49702-206,ABACAVIR SULFATE,600 mg/1,"TABLET, FILM COATED",600.0,mg,1,600.0,1.0,600.0,True,"[{'Number': [165], 'Unit': '°C'}, {'Number': [...","{'Number': [165], 'Unit': '°C'}",{'Number': [1.2]},165.0,1.2
abacavir,49702-217,ABACAVIR SULFATE,300 mg/1,"TABLET, FILM COATED",300.0,mg,1,300.0,1.0,300.0,True,"[{'Number': [165], 'Unit': '°C'}, {'Number': [...","{'Number': [165], 'Unit': '°C'}",{'Number': [1.2]},165.0,1.2


In [133]:
result_found.to_csv('210922_mol_screening_v1.csv')

In [141]:
x = finalData_df.MP.values
notfound_mols = finalData_df['Molecule'].values[np.argwhere(x!=x).reshape(-1)]

In [142]:
notfound_mols_set = set(list(notfound_mols))

In [143]:
len(notfound_mols)

165